# Formation Energy Lanscape of Cubic Perovskites

The formation energy ($\Delta E_{f}^{Pm\bar{3}m}$) of the perovskite structure is then calculated with respect to the energies of its constituent atoms as,
\begin{equation}
    \Delta E_f^{Pm\bar{3}m}\left(\mbox{A}\mbox{B}\mbox{X}_{3}\right)=\frac{1}{N}\left[ E_{ABX_{3}}-E_{A}-E_{B}-3E_{X} \right],
\end{equation}
in which $E$ represents the total  energy obtained from DFT, and is normalised by the total number of atoms $N$ in a simulation cell.

Our primary interest here is the relative stabilities of the cubic perovskite phase, which is governed by its convex-hull stabilities ($\Delta H_{c}$). However, this would require the knowledge of $\Delta E_{f}$ for the most stable phase for every compound with ABX$_{3}$ stoichiometry. This information is not available for all materials to be screened here, which is particularly true for hypothetical materials. Although performing structure predictions across different crystallographic space groups can help providing a more accurate evaluation on the relative phase stabilities,  it would be computationally too demanding to follow this route for every new compound investigated in this work. As such, following the approach of *J. Phys. Chem. A* **123**, 7323 (2019), we proximate $\Delta H_{c}$ as:
\begin{equation}
    \Delta H_c=\Delta E_f^{Pm\bar{3}m}-\min\left\{\Delta E_{f,i}^{\mbox{rand}}, i\in[1,10] \right\},
\end{equation}

Here, starting from the optimized structure of a cubic perovskite, 10 random crystal structures are generated by arbitrarily distorting its lattice parameters and atomic positions. The randomly generated structure is  then fully optimized without any constraints. The lowest energy within these 10 random structures gives the second term in the above equation. As such, $\Delta H_c$ effectively measures the convex hull stability of cubic perovskite with respect to a hypothetical $P1$ structure with the same composition. 

This notebook enable you to inspect the trend of $\Delta H_c$ for all perovskites stored in this database. The values of $\Delta H_c$ are plotted against the Goldschmidt tolerance factors for the cubic perovskites.

In [1]:
from tolerance_factor import tolerance_factor
from chemical_space import *
from ase.db import connect

Specify the energy landscape for which subset of compounds you would like to inspect, as categorized by the identity of the anion X in the ABX$_{3}$ perovskite.

In [2]:
X='F'

#get the corresponding A, B cations that go with the anion
if X in halide_C:
    A = halide_A
    B = halide_B
elif X in chalco_C:
    A = chalco_A
    B = chalco_B

#connect to the corresponding database toto retrieve the data
db=connect('./perovskites_'+str(X)+'.db')

In [3]:
tolerance_factors=[]
energy_differences=[] #as the formation energies of Pm3m perovskite to the most stable random structure

for a in A:
    for b in B:
        row = None
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        pm3m_formation_e = None
        
        tolerance_f = tolerance_factor(a, b, X)
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            randomised_formation_energies = []
            if pm3m_formation_e is not None:
                #Get the formation energies for all structures optimized from the starting structures constructed by
                #randomising the cubic perovskite structure
                for counter in range(10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    fe = None
                    row = None
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                            
                    if row is not None:
                        fe = row.key_value_pairs['formation_energy']
        
                        if fe is not None:
                            randomised_formation_energies.append(fe)

            if randomised_formation_energies != []:
                energy_differences.append(pm3m_formation_e - min(randomised_formation_energies))
                tolerance_factors.append(tolerance_f)

print("Total number of available data: "+str(len(energy_differences)))

Total number of available data: 491
